In [2]:
import os.path as osp
import random
import time

import cv2
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.optim as optim
import torch.utils.data as data

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [8]:
from utils.ssd_model import make_datapath_list,VOCDataset,DataTransform,Anno_xml2list,od_collate_fn


root_path="./data/VOCdevkit/VOC2012/"

train_img_list,train_anno_list,val_img_list,val_anno_list=make_datapath_list(root_path)

voc_classes=['aeroplane','bicycle','bird','boat','bottle','bus','car','cat','chair','cow','diningtable','dog','horse','motorbike','person','pottedplant','sheep','sofa','train','tvmonitor']
color_mean=(104,117,123)
input_size=300

train_dataset=VOCDataset(train_img_list,train_anno_list,phase="train",transform=DataTransform(input_size,color_mean),transform_anno=Anno_xml2list(voc_classes))
val_dataset=VOCDataset(val_img_list,val_anno_list,phase="val",transform=DataTransform(input_size,color_mean),transform_anno=Anno_xml2list(voc_classes))

batch_size=32
train_dataloader=data.DataLoader(train_dataset,batch_size=batch_size,shuffle=True,collate_fn=od_collate_fn)
val_dataloader=data.DataLoader(val_dataset,batch_size=batch_size,shuffle=False,collate_fn=od_collate_fn)
dataloaders_dict={"train":train_dataloader,"val":val_dataloader}


VGG 외 모듈의 초기 값으로 He를 사용
He: ReLU를 activation function인 경우 사용
- Kaiming He가 처음 제안해서 함수 명이 kaiming_normal_


In [12]:
from utils.ssd_model import SSD

ssd_cfg={
    'num_classes':21, #include background class
    'input_size':300, 
    'bbox_aspect_num':[4,6,6,6,4,4], #DBox 화면비 
    'feature_maps':[38,19,10,5,3,1], #각 source 별 화상 크기
    'steps':[8,16,32,64,100,300], 
    'min_sizes':[30,60,111,162,213,264], #DBox 최소 크기
    'max_sizes':[60,111,162,213,264,315],
    'aspect_ratios':[[2],[2,3],[2,3],[2,3],[2],[2]]
}

net=SSD(phase="train",cfg=ssd_cfg)

#load weight of vgg
vgg_weights=torch.load('./weights/vgg16_reducedfc.pth')
net.vgg.load_state_dict(vgg_weights)


#vgg 제외한 네트워크 He 초기화
def weights_init(m):
    if isinstance(m,nn.Conv2d):
        init.kaiming_normal_(m.weight.data)
        if m.bias is not None:
            nn.init.constant_(m.bias,0.0)

net.extras.apply(weights_init)
net.loc.apply(weights_init)
net.conf.apply(weights_init)

ModuleList(
  (0): Conv2d(512, 84, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): Conv2d(1024, 126, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (2): Conv2d(512, 126, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): Conv2d(256, 126, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (4): Conv2d(256, 84, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (5): Conv2d(256, 84, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
)

In [13]:
from utils.ssd_model import MultiBoxLoss

criterion=MultiBoxLoss(jaccard_thresh=0.5, neg_pos=3,device=device)

optimizer=optim.SGD(net.parameters(),lr=1e-3,momentum=0.9,weight_decay=5e-4)

In [32]:
def train_model(net,dataloaders_dict,criterion,optimizer,num_epochs):
    device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(device)
    net.to(device)

    torch.backends.cudnn.benchmark=True

    iteration=1
    epoch_train_loss=0.0
    epoch_val_loss=0.0
    logs=[]

    for epoch in range(num_epochs+1):
        t_epoch_start=time.time()
        t_iter_start=time.time()

        print(f"Epoch {epoch+1}/{num_epochs}")

        for phase in ['train','val']:
            if phase=="train":
                net.train()
                print("[train]")
            else:
                if (epoch+1)%10==0:
                    net.eval()
                    print("[val]")
                else:
                    continue
            
            for images,targets in dataloaders_dict[phase]:
                images=images.to(device)
                targets=[target.to(device) for target in targets]
                
                optimizer.zero_grad()
            
                with torch.set_grad_enabled(phase=='train'):
                    outputs=net(images)
                    loss_l,loss_c=criterion(outputs,targets)
                    loss=loss_l+loss_c
                    if phase=="train":
                        loss.backward()
                        
                        nn.utils.clip_grad_value_(net.parameters(),clip_value=2.0)

                        optimizer.step()

                        if iteration%10==0:
                            t_iter_finish=time.time()
                            print("[%s] loss:%.4f || sec:%.4f"%(iteration,loss.item(),t_iter_finish-t_iter_start))
                            t_iter_start=time.time()
                        
                        epoch_train_loss+=loss.item()
                        iteration+=1
                    else:
                        epoch_val_loss+=loss.item()
        t_epoch_finish=time.time()
        print("[%s] train_loss: %.4f || val_loss: %.4f || sec:%.4f"%(epoch+1,epoch_train_loss,epoch_val_loss,t_epoch_finish-t_epoch_start))
        t_epoch_start=time.time()

        log_epoch={'epoch':epoch+1,'train_loss':epoch_train_loss,'val_loss':epoch_val_loss}
        logs.append(log_epoch)
        df=pd.DataFrame(logs)
        df.to_csv("log_output.csv")

        epoch_train_loss=0.0
        epoch_val_loss=0.0
        if (epoch+1)%10==0:
            torch.save(net.state_dict(),f'weights/ssd300_{str(epoch+1)}.pth')

In [33]:
num_epochs=50
train_model(net,dataloaders_dict,criterion,optimizer,num_epochs)

cuda:0
Epoch 1/50
[train]
[10] loss:6.0245 || sec:6.2661
[20] loss:6.0080 || sec:6.1303
[30] loss:6.0311 || sec:6.5867
[40] loss:6.0588 || sec:6.2475
[50] loss:5.6995 || sec:6.5077
[60] loss:6.0300 || sec:6.1962
[70] loss:5.3181 || sec:6.2928
[80] loss:5.6360 || sec:6.4314
[90] loss:6.3560 || sec:6.4371
[100] loss:6.1098 || sec:6.3622
[110] loss:5.4636 || sec:6.1683
[120] loss:5.6870 || sec:6.3376
[130] loss:6.0577 || sec:6.2643
[140] loss:5.6380 || sec:6.3093
[150] loss:5.9108 || sec:6.1605
[160] loss:5.5446 || sec:6.2786
[170] loss:5.9732 || sec:6.4384
[1] train_loss: 1040.9310 || val_loss: 0.0000 || sec:115.6383
Epoch 2/50
[train]


/home/kist/nimod/pytorch deep learning/2_objectdetection/utils/data_augumentation.py:246: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  mode = random.choice(self.sample_options)
/home/kist/anaconda3/envs/pytorchDeepLearning/lib/python3.9/site-packages/torch/nn/functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"


[180] loss:5.5956 || sec:0.5501
[190] loss:5.9396 || sec:6.3746
[200] loss:5.7023 || sec:6.1061
[210] loss:5.7794 || sec:6.2572
[220] loss:5.8671 || sec:6.2486
[230] loss:5.4493 || sec:6.0414
[240] loss:5.4360 || sec:6.3778
[250] loss:5.8143 || sec:6.3506
[260] loss:6.1212 || sec:6.1737
[270] loss:5.3706 || sec:5.9689
[280] loss:5.4424 || sec:6.2701
[290] loss:5.2804 || sec:6.1841
[300] loss:5.1554 || sec:6.3776
[310] loss:5.5596 || sec:5.9340
[320] loss:5.2456 || sec:6.2193
[330] loss:5.6825 || sec:6.0958
[340] loss:5.5464 || sec:6.1965
[350] loss:5.3899 || sec:6.2433
[2] train_loss: 995.1711 || val_loss: 0.0000 || sec:113.5353
Epoch 3/50
[train]


/home/kist/nimod/pytorch deep learning/2_objectdetection/utils/data_augumentation.py:246: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  mode = random.choice(self.sample_options)
/home/kist/anaconda3/envs/pytorchDeepLearning/lib/python3.9/site-packages/torch/nn/functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"


[360] loss:5.6645 || sec:1.1514
[370] loss:5.5686 || sec:6.1808
[380] loss:5.0410 || sec:6.4073
[390] loss:5.6275 || sec:6.0394
[400] loss:5.5631 || sec:6.0560
[410] loss:5.2125 || sec:6.1981
[420] loss:5.3418 || sec:6.0568
[430] loss:5.7091 || sec:6.2164
[440] loss:5.2445 || sec:5.8504
[450] loss:4.9435 || sec:6.0098
[460] loss:5.5180 || sec:6.1093
[470] loss:5.2055 || sec:6.2019
[480] loss:4.9967 || sec:6.1259
[490] loss:5.0624 || sec:6.1362
[500] loss:4.8156 || sec:6.2375
[510] loss:5.3581 || sec:6.2228
[520] loss:5.4839 || sec:6.3969
[530] loss:5.0988 || sec:6.3188
[3] train_loss: 958.0563 || val_loss: 0.0000 || sec:112.9445
Epoch 4/50
[train]


...

[8600] loss:2.6977 || sec:5.0665
[8610] loss:3.2254 || sec:6.3970
[8620] loss:3.3420 || sec:6.7473
[8630] loss:3.1455 || sec:6.9091
[8640] loss:3.4714 || sec:6.6999
[8650] loss:3.0239 || sec:6.4818
[8660] loss:3.5376 || sec:6.7271
[8670] loss:3.4251 || sec:6.7428
[8680] loss:3.1605 || sec:6.2282
[8690] loss:3.2749 || sec:6.2326
[8700] loss:3.2758 || sec:6.3735
[8710] loss:3.2396 || sec:6.0962
[8720] loss:3.3945 || sec:6.4632
[8730] loss:3.5868 || sec:6.5860
[8740] loss:3.3422 || sec:6.5108
[8750] loss:3.0847 || sec:6.5707
[8760] loss:2.8948 || sec:6.2152
[8770] loss:3.9247 || sec:6.4159
[49] train_loss: 573.6177 || val_loss: 0.0000 || sec:118.7792
Epoch 50/50
[train]


/home/kist/nimod/pytorch deep learning/2_objectdetection/utils/data_augumentation.py:246: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  mode = random.choice(self.sample_options)
/home/kist/anaconda3/envs/pytorchDeepLearning/lib/python3.9/site-packages/torch/nn/functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"


[8780] loss:2.9761 || sec:5.7053
[8790] loss:3.1212 || sec:7.0228
[8800] loss:3.3139 || sec:6.5746
[8810] loss:3.5001 || sec:6.2853
[8820] loss:3.8449 || sec:6.3109
[8830] loss:2.7310 || sec:6.6707
[8840] loss:3.3010 || sec:6.8082
[8850] loss:3.3544 || sec:6.1615
[8860] loss:3.1523 || sec:6.2481
[8870] loss:3.3024 || sec:6.3466
[8880] loss:3.5311 || sec:6.0544
[8890] loss:2.8564 || sec:6.5258
[8900] loss:3.2463 || sec:6.2063
[8910] loss:3.1248 || sec:5.7846
[8920] loss:3.4167 || sec:5.8332
[8930] loss:3.5258 || sec:6.0985
[8940] loss:3.2958 || sec:6.1056
[8950] loss:3.3340 || sec:6.0474
[val]
[50] train_loss: 564.4648 || val_loss: 685.2678 || sec:160.8067
Epoch 51/50
[train]


/home/kist/nimod/pytorch deep learning/2_objectdetection/utils/data_augumentation.py:246: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  mode = random.choice(self.sample_options)
/home/kist/anaconda3/envs/pytorchDeepLearning/lib/python3.9/site-packages/torch/nn/functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"


[8960] loss:3.5695 || sec:5.9667
[8970] loss:3.2066 || sec:6.0838
[8980] loss:3.1546 || sec:6.0564
[8990] loss:2.1308 || sec:5.9291
[9000] loss:2.9469 || sec:5.9326
[9010] loss:3.2259 || sec:6.1366
[9020] loss:3.1442 || sec:6.2452
[9030] loss:2.6596 || sec:5.8490
[9040] loss:3.1345 || sec:6.0114
[9050] loss:3.1260 || sec:5.9705
[9060] loss:3.6270 || sec:6.1350
[9070] loss:2.8873 || sec:5.9470
[9080] loss:3.1046 || sec:6.1446
[9090] loss:3.3790 || sec:6.1641
[9100] loss:2.9047 || sec:5.8698
[9110] loss:3.3050 || sec:6.2267
[9120] loss:3.6422 || sec:6.0542
[51] train_loss: 564.7939 || val_loss: 0.0000 || sec:110.4696
